## Conv2D

In [1]:
import autoreload
%reload_ext autoreload
%autoreload 2

import sys
import os
import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
sys.path.append('../src')

import tensorflow as tf
from data_processing.mtb_data_provider import MtbDataProvider
from visualization.visualize import MtbVisualizer
from models.conv1d_model import Conv1dModel
from models.conv2d_model import Conv2dModel
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

In [2]:
RANDOM_SEED = 42

# 1. Set the `PYTHONHASHSEED` environment variable at a fixed value
import os
os.environ['PYTHONHASHSEED']=str(RANDOM_SEED)

# 2. Set the `python` built-in pseudo-random generator at a fixed value
import random
random.seed(RANDOM_SEED)

# 3. Set the `numpy` pseudo-random generator at a fixed value
import numpy as np
np.random.seed(RANDOM_SEED)

# 4. Set the `tensorflow` pseudo-random generator at a fixed value
import tensorflow as tf
tf.set_random_seed(RANDOM_SEED)

# 5. Configure a new global `tensorflow` session
from tensorflow.keras import backend as K
session_conf = tf.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)
sess = tf.Session(graph=tf.get_default_graph(), config=session_conf)
K.set_session(sess)

In [3]:
def load_data(window_size, raw_files=['neureuth', 'sachrang'], step_size=.25, frequency=40, dismiss_not_riding_label=False):
    X_result = None
    y_result = None
    for raw_file in raw_files:
        X_tmp, y_tmp = MtbDataProvider.load_data('../data/raw/' + raw_file, '*?[0-9].csv', frequency=frequency)
        X_tmp = MtbDataProvider.sync_sensors(X_tmp)
        X_tmp, y_tmp = MtbDataProvider.slice_sensors(X_tmp, y_tmp, 
                                                     window_size=window_size, 
                                                     frequency=frequency, 
                                                     step_size=step_size, 
                                                     dismiss_not_riding_label=dismiss_not_riding_label)

        if X_result is None:
            X_result = X_tmp
            y_result = y_tmp
        else:
            X_result = np.concatenate((X_result, X_tmp))
            y_result = np.concatenate((y_result, y_tmp))

    return X_result, y_result, window_size // frequency

In [ ]:
def run_experiments(i=0, epochs=300, batch_size=16, patience=50, frequency=40, dismiss_not_riding_label=False):
    window_sizes = [1000, 2000, 5000, 10000, 20000]
    kernel_lengths = [5, 10, 20, 40, 60]
    kernel_height = 2
    
    for window_size in window_sizes:
        X, y, sample_size = load_data(window_size, dismiss_not_riding_label=dismiss_not_riding_label)
        from sklearn.model_selection import train_test_split
        X_train, X_dev, y_train, y_dev = train_test_split(X, y, test_size=.2, random_state=RANDOM_SEED, shuffle=True)
        X_train, y_train = MtbDataProvider.evenly_oversample(X_train, y_train)
        
        for kernel_length in kernel_lengths:
            
            if (window_size // frequency < kernel_length):
                continue
            
            model_cnn2 = Conv2dModel.build_model(input_shape=X_train[0].shape, 
                                     cnn_blocks=3,
                                     dropout=0.3,
                                     kernel_size=(kernel_length, kernel_height),
                                     dismiss_not_riding_label=dismiss_not_riding_label)
            
            filepath="../checkpoints/conv2d-" + str(window_size) + "-" + str(kernel_length) + "-{epoch:02d}-{val_sparse_categorical_accuracy:.4f}.hdf5"
            
            checkpoint = ModelCheckpoint(filepath, save_best_only = True, monitor='val_sparse_categorical_accuracy', mode='max')
            early_stopping = EarlyStopping(monitor='val_sparse_categorical_accuracy', min_delta=0, patience=patience, verbose=0, mode='auto', baseline=None, restore_best_weights=False)
            
            class_weight = {
                0: 100.,
                1: 100.,
                2: 100.,
                3: 100.
            }
            
            model_cnn2.fit(X_train, y_train,
              epochs=epochs,
              class_weight=class_weight,
              batch_size=batch_size,
              validation_data=(X_dev, y_dev), 
              callbacks = [checkpoint, early_stopping])
            
            img_base_path = "../checkpoints/img/conv2d-" + str(i) + "-" + str(window_size) + "-" + str(kernel_length) + "-"
            img_path_ending = ".png"

            loss_plot_path = img_base_path  + "accuracy_plot" + img_path_ending
            plt.figure(i)
            plt.plot(model_cnn2.history.history['sparse_categorical_accuracy'])
            plt.plot(model_cnn2.history.history['val_sparse_categorical_accuracy'])
            plt.xlabel('epoch')
            plt.ylabel('sca')
            plt.savefig(loss_plot_path, dpi=300)
            plt.close()
            
            y_pred_dev = model_cnn2.predict(X_dev)
            y_pred_dev = np.argmax(y_pred_dev, axis=1)

            X_dev_plot = X_dev if len(X_dev.shape) <= 3 else X_dev[:,:,0]
            pred_file_path = img_base_path  + "pred_plot" + img_path_ending
            #MtbVisualizer.plot_results(X_dev_plot, y_pred_dev, sample_size, pred_file_path)
            
            act_file_path = img_base_path  + "actual_plot" + img_path_ending
            #MtbVisualizer.plot_results(X_dev_plot, y_dev, sample_size, act_file_path)
            
            confusion_path = img_base_path  + "confusion_matrix" + img_path_ending
            labels = [0,1,2,3] if not dismiss_not_riding_label else [0,1,2]
            print(labels)
            MtbVisualizer.print_confusion_matrix(y_dev, y_pred_dev, labels, confusion_path)


In [ ]:
run_experiments(i=0, epochs=1500, batch_size=32, dismiss_not_riding_label=True, patience=250)

reading  ../data/raw/neureuth_acc_1.csv
reading  ../data/raw/neureuth_acc_2.csv
reading  ../data/raw/neureuth_gyr_1.csv
reading  ../data/raw/neureuth_gyr_2.csv
reading  ../data/raw/sachrang_acc_1.csv
reading  ../data/raw/sachrang_acc_2.csv
reading  ../data/raw/sachrang_gyr_1.csv
reading  ../data/raw/sachrang_gyr_2.csv
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Train on 10368 samples, validate on 1485 samples
Epoch 1/1500
10368/10368 [==============================] - 3s 333us/sample - loss: 1.3835 - sparse_categorical_accuracy: 0.3690 - val_loss: 1.2912 - val_sparse_categorical_accuracy: 0.3293
Epoch 2/1500
10368/10368 [==============================] - 2s 176us/sample - loss: 1.2445 - sparse_categorical_accuracy: 0.4011 - val_loss: 1.2613 - val_sparse_categorical_accuracy: 0.3293
Epoch 3/1500
10368/10368 [==============================] - 2s 1

Epoch 81/1500
10368/10368 [==============================] - 2s 190us/sample - loss: 0.8946 - sparse_categorical_accuracy: 0.5818 - val_loss: 0.9910 - val_sparse_categorical_accuracy: 0.4519
Epoch 82/1500
10368/10368 [==============================] - 2s 196us/sample - loss: 0.9051 - sparse_categorical_accuracy: 0.5834 - val_loss: 0.9990 - val_sparse_categorical_accuracy: 0.4505
Epoch 83/1500
10368/10368 [==============================] - 2s 195us/sample - loss: 0.8979 - sparse_categorical_accuracy: 0.5802 - val_loss: 0.9676 - val_sparse_categorical_accuracy: 0.4828
Epoch 84/1500
10368/10368 [==============================] - 2s 191us/sample - loss: 0.8940 - sparse_categorical_accuracy: 0.5846 - val_loss: 1.0034 - val_sparse_categorical_accuracy: 0.4492
Epoch 85/1500
10368/10368 [==============================] - 2s 191us/sample - loss: 0.9048 - sparse_categorical_accuracy: 0.5765 - val_loss: 0.9630 - val_sparse_categorical_accuracy: 0.4842
Epoch 86/1500
10368/10368 [==================

Epoch 124/1500
10368/10368 [==============================] - 2s 181us/sample - loss: 0.8968 - sparse_categorical_accuracy: 0.5845 - val_loss: 0.9810 - val_sparse_categorical_accuracy: 0.4727
Epoch 125/1500
10368/10368 [==============================] - 2s 190us/sample - loss: 0.9022 - sparse_categorical_accuracy: 0.5800 - val_loss: 1.0071 - val_sparse_categorical_accuracy: 0.4276
Epoch 126/1500
10368/10368 [==============================] - 2s 189us/sample - loss: 0.8958 - sparse_categorical_accuracy: 0.5857 - val_loss: 0.9944 - val_sparse_categorical_accuracy: 0.4438
Epoch 127/1500
10368/10368 [==============================] - 2s 190us/sample - loss: 0.8946 - sparse_categorical_accuracy: 0.5852 - val_loss: 1.0073 - val_sparse_categorical_accuracy: 0.4411
Epoch 128/1500
10368/10368 [==============================] - 2s 192us/sample - loss: 0.8993 - sparse_categorical_accuracy: 0.5891 - val_loss: 0.9785 - val_sparse_categorical_accuracy: 0.4667
Epoch 129/1500
10368/10368 [============

Epoch 167/1500
10368/10368 [==============================] - 2s 190us/sample - loss: 0.8960 - sparse_categorical_accuracy: 0.5856 - val_loss: 1.0416 - val_sparse_categorical_accuracy: 0.4121
Epoch 168/1500
10368/10368 [==============================] - 2s 188us/sample - loss: 0.8956 - sparse_categorical_accuracy: 0.5879 - val_loss: 1.0184 - val_sparse_categorical_accuracy: 0.4465
Epoch 169/1500
10368/10368 [==============================] - 2s 191us/sample - loss: 0.8992 - sparse_categorical_accuracy: 0.5903 - val_loss: 1.0015 - val_sparse_categorical_accuracy: 0.4391
Epoch 170/1500
10368/10368 [==============================] - 2s 191us/sample - loss: 0.8979 - sparse_categorical_accuracy: 0.5880 - val_loss: 1.0754 - val_sparse_categorical_accuracy: 0.3791
Epoch 171/1500
10368/10368 [==============================] - 2s 187us/sample - loss: 0.8953 - sparse_categorical_accuracy: 0.5896 - val_loss: 1.0473 - val_sparse_categorical_accuracy: 0.4141
Epoch 172/1500
10368/10368 [============

In [9]:
from sklearn.model_selection import train_test_split
window_sizes = [1000, 2000, 5000, 10000, 20000]

for window_size in window_sizes:
    X, y, sample_size = load_data(window_size, dismiss_not_riding_label=True)
    X_train, X_dev, y_train, y_dev = train_test_split(X, y, test_size=.2, random_state=RANDOM_SEED, shuffle=True)
    X_train_o, y_train_o = MtbDataProvider.evenly_oversample(X_train, y_train)
    print("y_train", len(y_train))
    print("y_train_o", len(y_train_o))
    print("y_dev", len(y_dev))

reading  ../data/raw/neureuth_acc_1.csv
reading  ../data/raw/neureuth_acc_2.csv
reading  ../data/raw/neureuth_gyr_1.csv
reading  ../data/raw/neureuth_gyr_2.csv
reading  ../data/raw/sachrang_acc_1.csv
reading  ../data/raw/sachrang_acc_2.csv
reading  ../data/raw/sachrang_gyr_1.csv
reading  ../data/raw/sachrang_gyr_2.csv
y_train 5937
y_train_o 10368
y_dev 1485
reading  ../data/raw/neureuth_acc_1.csv
reading  ../data/raw/neureuth_acc_2.csv
reading  ../data/raw/neureuth_gyr_1.csv
reading  ../data/raw/neureuth_gyr_2.csv
reading  ../data/raw/sachrang_acc_1.csv
reading  ../data/raw/sachrang_acc_2.csv
reading  ../data/raw/sachrang_gyr_1.csv
reading  ../data/raw/sachrang_gyr_2.csv
y_train 2971
y_train_o 5073
y_dev 743
reading  ../data/raw/neureuth_acc_1.csv
reading  ../data/raw/neureuth_acc_2.csv
reading  ../data/raw/neureuth_gyr_1.csv
reading  ../data/raw/neureuth_gyr_2.csv
reading  ../data/raw/sachrang_acc_1.csv
reading  ../data/raw/sachrang_acc_2.csv
reading  ../data/raw/sachrang_gyr_1.csv
re